In [1]:
#Preparing data into Graphs

In [1]:
import numpy as np
import geopandas as gpd
import pandas as pd
from scipy.spatial import cKDTree,distance
from stellargraph import StellarGraph
from tqdm.notebook import tqdm

/usr/local/lib/python3.8/dist-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.9.1-CAPI-1.14.2) is incompatible with the GEOS version PyGEOS was compiled with (3.10.0-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(
2022-02-09 06:53:21.556795: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/lib::/usr/lib/jvm/java-11-openjdk-amd64/lib/server:/opt/hadoop/lib/native:/usr/local/lib/R/lib:/usr/local/grass78/lib
2022-02-09 06:53:21.556843: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-02-09 06:53:24.951260: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such

In [290]:
def getNeighbours(in_file,geom_col='geometry'):
    in_df=gpd.read_file(in_file).set_index('cat')
    in_df=in_df.to_crs('EPSG:32645')
    #return in_df
    for index, row in tqdm(in_df.iterrows(),total=in_df.shape[0]):
        #print(index)
        neighbors = in_df[in_df.geometry.touches(row[geom_col])].index.tolist() 
        nA=np.array(list(in_df['geometry'][[index]].centroid.apply(lambda x: (x.x, x.y))))
        nB = np.array(list(in_df['geometry'][neighbors].centroid.apply(lambda x: (x.x, x.y))))
        try:
            dist=distance.cdist(nA,nB,metric='euclidean').tolist()[0]
        except:
            print(f'Neigbours not found for {index}')
            in_df=in_df.drop(index)
            continue
        #print(neighbors[0],dist)
        #print(dist)
        for i in range (len(dist)):
            nb_col='nb_'+str(i+1)
            ds_col='ds_'+str(i+1)
            if nb_col not in in_df:
                in_df[nb_col] = ""
                in_df[ds_col] = ""
            in_df.at[index, nb_col] = neighbors[i]
            in_df.at[index, ds_col] =  dist[i]
    return in_df
def getNNearNeighbours(in_file,id_col,n_neighbour=5,geom_col='geometry'):
    in_df=gpd.read_file(in_file)
    in_df=in_df.to_crs('EPSG:32645')
    nA = np.array(list(in_df[geom_col].centroid.apply(lambda x: (x.x, x.y))))
    tree = cKDTree(nA)
    dist, idx = tree.query(nA, k=n_neighbour+1)
    for i in range(1,n_neighbour+1):
        nb_col='nb_'+str(i)
        ds_col='ds_'+str(i)
        in_df[nb_col]=idx[:,i]
        in_df[ds_col]=dist[:,i]
    return in_df


    

In [291]:
in_df='Data/SlopeUnits/SlopeUnits_V3.shp'
nnear=getNeighbours(in_df)
#alln=getNeighbours(in_df)

  0%|          | 0/58052 [00:00<?, ?it/s]

Neigbours not found for 147678
Neigbours not found for 264817
Neigbours not found for 268620
Neigbours not found for 175366
Neigbours not found for 267233
Neigbours not found for 375777
Neigbours not found for 375828
Neigbours not found for 372659
Neigbours not found for 146272
Neigbours not found for 252447
Neigbours not found for 266798
Neigbours not found for 260691
Neigbours not found for 146176
Neigbours not found for 146069
Neigbours not found for 376000
Neigbours not found for 260581
Neigbours not found for 301175
Neigbours not found for 249794
Neigbours not found for 375801
Neigbours not found for 375938
Neigbours not found for 252407
Neigbours not found for 375839
Neigbours not found for 232013
Neigbours not found for 375723
Neigbours not found for 375457
Neigbours not found for 146076
Neigbours not found for 240086
Neigbours not found for 241244
Neigbours not found for 190529
Neigbours not found for 374586
Neigbours not found for 354575
Neigbours not found for 375738
Neigbour

In [292]:
nnear=nnear.drop(columns=['geometry','Area'])

In [293]:
nnear.to_csv('Data/nodes/neighbourhood.csv')

### load the excel sheet from pandas and join

# prepare Feature data

In [294]:
neighbourhood_data=pd.read_csv('Data/nodes/neighbourhood.csv')
covars=pd.read_excel('Data/Covars_yearly/all_covars.xls')

In [ ]:
covars.columns.tolist()

In [10]:
constant_covars=covars[['Cat', 'clay','organic', 'HorizontalCurvature_mean','HorizontalCurvature_stdDev','VerticalCurvature_mean','VerticalCurvature_stdDev',
                 'aspect_mean','aspect_stdDev','elevation_mean','elevation_stdDev','slope_mean','slope_stdDev']]

In [11]:
covars=covars.drop(columns=[ 'clay','organic', 'HorizontalCurvature_mean','HorizontalCurvature_stdDev','VerticalCurvature_mean','VerticalCurvature_stdDev',
                 'aspect_mean','aspect_stdDev','elevation_mean','elevation_stdDev','slope_mean','slope_stdDev'])

In [44]:
 #0_NdMx_mean,0_NdMx_stdDev 'b1_PMx' 'b7_PMe' 'b6_PSt'
for i in tqdm(range(31)):
    ndvi_mean_col=f'{str(i)}_NdMx_mean'
    ndvi_stdv_col=f'{str(i)}_NdMx_stdDev'
    prec_mean_col=f'b{str(i+1)}_PMx'
    prec_stdv_col=f'b{str(i+1)}_PMe'
    prec_maxi_col=f'b{str(i+1)}_PSt'
    covars_subset=covars[['Cat',ndvi_mean_col,ndvi_stdv_col,prec_mean_col,prec_stdv_col,prec_maxi_col]]
    covar_data=pd.merge(constant_covars,covars_subset,left_on='Cat',right_on='Cat',left_index=False)
    covar_data.to_csv(f'Data/covars_{str(1988+i)}.csv')
    #break

  0%|          | 0/31 [00:00<?, ?it/s]

In [296]:
df_neighbour=pd.DataFrame(columns=['target','source','weight'])
for i in tqdm(range(1,26)):
    nb_col=f'nb_{str(i)}'
    ds_col=f'ds_{str(i)}'
    df_subset=neighbourhood_data[['cat',nb_col,ds_col]].dropna()
    df_subset=df_subset.rename(columns={'cat':'target',nb_col:'source',ds_col:'weight'})
    df_neighbour=df_neighbour.append(df_subset, ignore_index=True)
    #break

  0%|          | 0/25 [00:00<?, ?it/s]

In [297]:
df_neighbour

,target,source,weight
0,328069,328157,1465.229143
1,330938,330395,1059.508589
2,258925,257922,527.723837
3,322424,322391,1371.401816
4,216685,215877,504.676123
...,...,...,...
341748,354815,355215.0,1272.979159
341749,298406,291148.0,4499.337638
341750,354815,354795.0,2148.723794
341751,298406,296211.0,806.419163


In [298]:
df_neighbour.to_csv('Data/node_matrix.csv')

# Get target variables

In [6]:
import geopandas as gpd
import pandas as pd
from tqdm.notebook import tqdm
df_inventory=gpd.read_file('Data/Inventory/DataFile5_-_12920ASMinventory.shp')
df_slopeunit=gpd.read_file('Data/SlopeUnits/SlopeUnits_V3.shp')
df_slopeunit=df_slopeunit.to_crs(df_inventory.crs)

In [43]:
df_inventory['area_inventory']=df_inventory.area
df_slopeunit['area_slopeunit']=df_slopeunit.area
for i in tqdm(range(31)):
    df_slopeunit_sub=df_slopeunit
    monsoon_year=1988+i
    inventory_subset=df_inventory[df_inventory['Monsoon_Ye']==str(monsoon_year)]
    if inventory_subset.empty:
        print(f'no data exists for {str(monsoon_year)}')
        continue
    sjoined=gpd.overlay(inventory_subset, df_slopeunit, how='intersection')
    sjoined['area_new']=sjoined.area
    sjoined=sjoined.groupby(['cat'])['area_new'].agg(['sum','count'])
    sjoined=sjoined.rename(columns={'sum':'area_inventory','count':'number_landslide'})
    df_slopeunit_sub['landslide']=0
    df_slopeunit_sub=df_slopeunit_sub.join(sjoined, on='cat')
    df_slopeunit_sub.loc[df_slopeunit_sub['area_inventory']>0,'landslide']=1
    df_slopeunit_sub=df_slopeunit_sub.fillna(0)
    df_slopeunit_sub['area_density']=100*(df_slopeunit_sub.area_inventory/df_slopeunit_sub.area_slopeunit)
    df_slopeunit_sub=df_slopeunit_sub[['cat','area_density','number_landslide','landslide']]
    file_name=f'Data/Targets/targets_{str(monsoon_year)}.csv'
    if (df_slopeunit_sub[df_slopeunit_sub.area_density>100].empty==False):
        print('ok')
        break
    df_slopeunit_sub.to_csv(file_name)
    

  0%|          | 0/31 [00:00<?, ?it/s]

no data exists for 2011
no data exists for 2012


In [37]:
df_slopeunit_sub=df_slopeunit


,area_inventory,number_landslide
cat,,
183932,29399.699971,1
185758,29399.699971,1
197836,26769.770230,1
198533,9752.518020,1
198653,206380.625175,1
...,...,...
372444,32878.315809,1
372626,7240.437164,1
372713,32878.315809,1


In [41]:
df_slopeunit_sub[df_slopeunit_sub.area_density>0]

,Area,cat,geometry,area_slopeunit,landslide,area_inventory,number_landslide,area_density
130,670920.625,306108,"POLYGON ((489569.989 3047790.125, 489584.881 3...",6.704184e+05,1,7927.753158,1.0,1.182508
423,2783107.801,333450,"POLYGON ((580289.788 3023415.144, 580305.174 3...",2.781231e+06,1,501689.009675,8.0,18.038378
614,1982436.676,333866,"POLYGON ((575879.900 3022200.000, 575879.902 3...",1.981290e+06,1,17570.597082,1.0,0.886826
618,379580.919,199681,"POLYGON ((290009.876 3127199.795, 290024.764 3...",3.797988e+05,1,25446.262486,1.0,6.699933
731,509786.566,325220,"POLYGON ((458339.938 3029820.000, 458354.827 3...",5.093908e+05,1,337.771702,1.0,0.066309
...,...,...,...,...,...,...,...,...
57362,1137623.648,199267,"POLYGON ((310679.838 3126899.978, 310695.228 3...",1.137700e+06,1,16229.873560,1.0,1.426551
57454,739735.243,199357,"POLYGON ((309270.106 3127529.883, 309270.105 3...",7.397870e+05,1,2380.876225,1.0,0.321833
57599,690611.993,365574,"POLYGON ((469230.125 2978909.776, 469245.019 2...",6.899061e+05,1,7547.614057,1.0,1.094006
57676,585834.162,239666,"POLYGON ((329609.973 3099599.798, 329624.860 3...",5.858193e+05,1,6191.908228,1.0,1.056966


# create Graphs

In [299]:
graphs=[]
nodes=pd.read_csv('Data/node_matrix.csv',index_col=0)
nodes['source']=nodes['source'].astype(int)
for i in tqdm(range(31)):
    features_file=f'Data/covars_{str(1988+i)}.csv'
    features_data=pd.read_csv(f'Data/Covars_yearly/covars_{str(1988+i)}.csv',index_col=0).set_index('Cat')
    graph = StellarGraph(features_data,nodes)
    graphs.append(graph)

  0%|          | 0/31 [00:00<?, ?it/s]

In [300]:
graphs[0].info()

'StellarGraph: Undirected multigraph\n Nodes: 58052, Edges: 341753\n\n Node types:\n  default: [58052]\n    Features: float32 vector, length 17\n    Edge types: default-default->default\n\n Edge types:\n    default-default->default: [341753]\n        Weights: range=[25.4536, 7685.69], mean=1176.85, std=631.384\n        Features: none'

In [301]:
features_data

,clay,organic,HorizontalCurvature_mean,HorizontalCurvature_stdDev,VerticalCurvature_mean,VerticalCurvature_stdDev,aspect_mean,aspect_stdDev,elevation_mean,elevation_stdDev,slope_mean,slope_stdDev,30_NdMx_mean,30_NdMx_stdDev,b31_PMx,b31_PMe,b31_PSt
Cat,,,,,,,,,,,,,,,,,
330938,13.804274,2.814837,0.000041,0.000407,-0.000105,0.000636,235.653516,80.420060,225.068661,7.989450,3.070058,1.971146,0.681430,0.015650,48.221889,4.967041,9.871718
332460,12.649948,2.509876,0.000017,0.000312,-0.000004,0.000369,221.416476,58.947243,361.711077,19.873541,2.296744,1.377863,0.648912,0.048879,65.372414,5.016054,10.206280
329268,14.714447,2.672427,-0.000107,0.002005,-0.000067,0.000993,170.238349,52.932861,536.215113,39.208104,9.380708,3.080571,0.547350,0.040670,72.739525,5.088988,11.147287
322762,15.926574,3.225687,0.000108,0.001296,0.000864,0.001609,215.525700,92.256204,459.530459,22.963471,5.706772,3.802420,0.683454,0.011223,79.915848,5.111228,11.053352
300883,14.673804,2.723728,0.000079,0.002673,-0.000049,0.001786,245.884936,49.092416,606.852486,160.803574,20.977737,8.728044,0.639748,0.076018,89.450447,5.157235,10.898296
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
251087,14.481656,5.373497,-0.000038,0.000450,-0.000268,0.000688,75.516247,79.430815,4572.506878,18.360539,5.531359,3.849466,0.371008,0.109263,54.664452,3.639956,8.720836
245589,12.910275,5.519663,0.000054,0.001118,-0.000145,0.001021,112.875184,67.618628,5016.070500,67.911834,8.127049,3.511709,0.195880,0.049551,43.528488,3.315684,8.145844
251122,11.563867,5.689320,-0.001868,0.002157,-0.000302,0.001140,209.513123,19.575896,5736.096591,107.709969,51.861079,10.205617,0.336029,0.002939,72.271957,4.020911,9.855270
